# Introduction
In This file, I will train a Yolov11 model to detect wall cracks and road cracks, The project will use the Yolov11 segmentation large model version that is previously trained on the COCO dataset. However, I will retrain it on the custom dataset to detect wall and road cracks.

# 1. Import libraries

In [1]:
from ultralytics import YOLO
import torch

In [2]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

# 2. Importing the custom cracks dataset

- Dataset Name: Crack Computer Vision Dataset
- URL: https://universe.roboflow.com/apu-jimbr/crack-uuasm

- Dataset Description: The dataset has about 4000 images of cracks in roads, walls, and cement in general.
1. The Number of files in data\test\images is 322.
2. The Number of files in data\test\labels is 322.
3. The Number of files in data\train\images is 7072.
4. The Number of files in data\train\labels is 7072.
5. The Number of files in data\valid\images is 455.
6. The Number of files in data\valid\labels is 455.

In [3]:
def get_directory(directory, structure=['images', 'labels']):
    folders = os.listdir(directory)
    for folder in folders:
        folder_path = os.path.join(directory, folder)
        if os.path.isdir(folder_path):
            subfolders = os.listdir(folder_path)
            for final_folders in subfolders:
                if final_folders in structure:
                    final_folder_path = os.path.join(folder_path, final_folders)
                    files = [f for f in os.listdir(final_folder_path) if os.path.isfile(os.path.join(final_folder_path, f))]
                    num_files = len(files)
                    print(f"The Number of files in {final_folder_path} is {num_files}.")

get_directory("data")

The Number of files in data\test\images is 322.
The Number of files in data\test\labels is 322.
The Number of files in data\train\images is 14144.
The Number of files in data\train\labels is 7072.
The Number of files in data\valid\images is 910.
The Number of files in data\valid\labels is 455.


# 3. Training the model

In [4]:
model = YOLO('yolo11n-seg.pt')

In [5]:
# Train Settings
coco_path = "data/data.yaml"
epochs = 25
input_image_size = 640
patience = 3
batch = 32
val = True
cache = True
cache= 'disk'
plots = True
project = "Yolov11-test"
workers = 2

In [6]:
# Clear GPU Cache
torch.cuda.empty_cache()

In [7]:
# Train the Model
results = model.train(data=coco_path, epochs=epochs, imgsz=input_image_size, patience=patience, batch=batch, val=val, cache=cache, plots=plots, project=project, workers=workers)

Ultralytics 8.3.228  Python-3.13.5 torch-2.8.0+cu129 CUDA:0 (NVIDIA GeForce RTX 2080 SUPER, 8192MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=disk, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=25, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=3, perspective=0.0, plots=True, p

2025/11/18 16:30:36 INFO mlflow.tracking.fluent: Experiment with name 'Yolov11-test' does not exist. Creating a new experiment.


MLflow: logging run_id(8c85230252444f74878c86f38ad62aac) to runs\mlflow
MLflow: view at http://127.0.0.1:5000 with 'mlflow server --backend-store-uri runs\mlflow'
MLflow: disable with 'yolo settings mlflow=False'
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to G:\Work Projects\AI & ML Projects\AI-ML-Projects\Yolov11-Wall & Road Cracks-Detection\Yolov11-test\train
Starting training for 25 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size
       1/25      5.62G      1.475      3.042      2.523      1.523         90        640: 100% ━━━━━━━━━━━━ 221/221 2.0s/it 7:31<1.3s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 8/8 1.1it/s 7.3s0.8ss
                   all        455        780      0.203      0.258      0.106     0.0352       0.15      0.169     0.0468     0.0121

      Epoch    GPU_mem   box_loss   se

# 4. Inference Testing

In [27]:
import cv2
import glob
import matplotlib.pyplot as plt

In [ ]:
# Function to read all images present in the folder
def read_images_from_folder(folder_path):
    """
    Read all images present in the folder
    """
    images = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image_path = os.path.join(folder_path, filename)
            image = cv2.imread(image_path)
            images.append(image)
    return images

# Inference Folder Path
inference_folder_path = "inference images"

# Read all images from the folder
images = read_images_from_folder(inference_folder_path)

In [31]:
def batch_inference_save_results(model, image_paths, output_dir, batch_size=8):
    """
    Perform batch inference and save results to files
    """
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)
    
    # Run inference and save results
    results = model(image_paths, 
                   save=True,           
                   save_txt=True,       
                   save_conf=True,      
                   project=output_dir,  
                   batch=batch_size)    
    
    return results

# Load model
model = YOLO(r"G:\Work Projects\AI & ML Projects\AI-ML-Projects\Yolov11-Wall & Road Cracks-Detection\Yolov11-test\train\weights\best.pt")

# Run batch inference and save
results = batch_inference_save_results(model, images, output_dir="inference images/results", batch_size=4)
print(f"Results saved for {len(results)} images")


0: 640x640 1 crack, 2.8ms
1: 640x640 1 crack, 2.8ms
2: 640x640 1 crack, 2.8ms
3: 640x640 1 crack, 2.8ms
4: 640x640 (no detections), 2.8ms
5: 640x640 2 cracks, 2.8ms
6: 640x640 1 crack, 2.8ms
7: 640x640 2 cracks, 2.8ms
8: 640x640 2 cracks, 2.8ms
9: 640x640 1 crack, 2.8ms
Speed: 1.9ms preprocess, 2.8ms inference, 1.5ms postprocess per image at shape (4, 3, 640, 640)
Results saved to G:\Work Projects\AI & ML Projects\AI-ML-Projects\Yolov11-Wall & Road Cracks-Detection\inference images\results\predict
9 labels saved to G:\Work Projects\AI & ML Projects\AI-ML-Projects\Yolov11-Wall & Road Cracks-Detection\inference images\results\predict\labels
Results saved for 10 images
